In [19]:
import nltk

import numpy as np

import pandas as pd

import sklearn as sk

import csspipe
import cssfeature

import helper

import sqlite3



In [20]:
### Load data

# (run bash ./src/init.sh first!)

In [21]:
#data = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
#help(data.head)
#data.head(1)
db_path = './data.sqlite3'
sqlite_connection = sqlite3.connect(db_path)
data_reader = helper.CNeuReader(sqlite_connection)

data = data_reader.get_results()
data.head()


,sentence,cNeu
0,likes the sound of thunder.,True
1,is so sleepy it's not even funny that's she ca...,True
2,is sore and wants the knot of muscles at the b...,True
3,likes how the day sounds in this new song.,True
4,is home. <3,True


In [22]:
#help(data)

In [23]:

print(data.head(2))

cssfeature.CharacterFeatures.string_length(data, 0)

print(data.head(2))


                                            sentence  cNeu
0                        likes the sound of thunder.  True
1  is so sleepy it's not even funny that's she ca...  True
  sentence  cNeu
0       27  True
1       63  True


In [6]:
### Split Data in train and test sets
# http://scikit-learn.org/stable/modules/cross_validation.html

In [44]:

test_data = neu[["STATUS"]]
test_target = neu[["cNEU"]]

split = sk.cross_validation.train_test_split(test_data, test_target, train_size = 0.66, random_state= 6432119)
train_feat, test_feat, train_label, test_label = split

print("train_feat\n%s\n" %train_feat.head(1))
print("train_label\n%s\n" %train_label.head(1))
print("test_feat\n%s\n" %test_feat.head(1))
print("test_label\n%s\n" %test_label.head(1))



train_feat
                                                 STATUS
8771  Merry Christmas & All The Love...  *PROPNAME* ...

train_label
     cNEU
8771    n

test_feat
                                                 STATUS
7586  Cos all of the stars are fading away Just try ...

test_label
     cNEU
7586    n



In [8]:
### Create classifiers

In [9]:
## Support Vector Machines
# http://scikit-learn.org/stable/modules/svm.html

classifier_svc = ("support vector classifier", sk.svm.SVC(kernel='linear', C=1))
classifier_l_svc = ("linear svc", sk.svm.LinearSVC(random_state = 9974))


In [10]:
## Naive Bayes
# http://scikit-learn.org/stable/modules/naive_bayes.html
classifier_m_nb = ("Naive Bayes classifier for multinomial models", sk.naive_bayes.MultinomialNB())
classifier_b_nb = ("Naive Bayes classifier for multivariate Bernoulli models", sk.naive_bayes.BernoulliNB())

In [11]:
## Ensemble methods
# http://scikit-learn.org/stable/modules/ensemble.html
classifier_rfc = ("random forest classifier",
                  sk.ensemble.RandomForestClassifier(
                        max_depth=5
                        , n_estimators=10 
                        , max_features=1 
                        , n_jobs=-1
                        , random_state=32414343))
classifier_abc = ("Ada boost classifier", sk.ensemble.AdaBoostClassifier(random_state=67294))


In [12]:
## Neighbour methods
# http://scikit-learn.org/stable/modules/neighbors.html

classifier_knc = ("K nearest keighbours", sk.neighbors.KNeighborsClassifier())

In [13]:
## Pipelineing
# http://scikit-learn.org/stable/modules/pipeline.html
# http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

pipeline = sk.pipeline.Pipeline([
        classifier_l_svc
    ])


pipeline

Pipeline(steps=[('linear svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=9974, tol=0.0001,
     verbose=0))])

In [14]:

classifier = classifier_svc[1]

#help(train_feat["StringLength"])

sk.cross_validation.cross_val_score(classifier, train_feat[["StringLength"]], train_label["cNEU"])


#help(classifier)

#trained_classifier = classifier.fit(train_feat, train_label)

#predicted = sk.cross_validation.cross_val_predict(trained_classifier, test_feat, test_label)

#print(predicted)

#predicted = sk.cross_validation.cross_val_predict(pipeline, train_feat, test_label, cv=10)
#metrics.accuracy_score(iris.target, predicted) 

array([ 0.62482822,  0.62494269,  0.62494269])

In [32]:
tfidf_vectorizer = sk.feature_extraction.text.TfidfVectorizer()

pipeline = sk.pipeline.Pipeline([('tfidf', tfidf_vectorizer),
                     ('chi2', sk.feature_selection.SelectKBest(sk.feature_selection.chi2, k=1000)),
                     ('nb', sk.naive_bayes.MultinomialNB())
                    ])

#pipeline.fit(train_feat[["STATUS"]], train_label[["cNEU"]])
data = train_feat.STATUS
target = train_label.cNEU.apply(lambda x: x == 'y')

foo = tfidf_vectorizer.fit_transform(data[:1], target[:1])
print(foo)
#idf = foo.idf_

#print(len(idf))
#print(len(data))

result = sk.cross_validation.cross_val_score(pipeline, data, target)
print(result)

  (0, 6)	0.377964473009
  (0, 4)	0.377964473009
  (0, 2)	0.377964473009
  (0, 5)	0.377964473009
  (0, 0)	0.377964473009
  (0, 1)	0.377964473009
  (0, 3)	0.377964473009
[ 0.63078333  0.62998624  0.62906923]


In [16]:
pipe_factory = csspipe.PipeFactory()


In [17]:
tfidf_nb_pipe = pipe_factory.create_tfidf_nb()

In [18]:
tfidf_nb_pipe

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...yes classifier for multinomial models', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [22]:
result = sk.cross_validation.cross_val_score(tfidf_nb_pipe, data, target)
print(result)

[ 0.63078333  0.62998624  0.62906923]
